In [1]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\Deep_learning\\Deep-learning-model'

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluateConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size:int



In [5]:
from Hand_gesture_clf.constants import *
from Hand_gesture_clf.utils.common import read_yaml, create_diorectories, save_json



In [18]:
class ConfigManager:
    def __init__(self,
                 config_file_path  =CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params  =read_yaml(params_file_path)

        create_diorectories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluateConfig:
        eval_config = EvaluateConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data= "artifacts\data_ingestion\HandGesture\images",
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size= 100
        )

        return eval_config

In [7]:
from urllib.parse import urlparse

In [19]:
class Evaluation:
    def __init__(self, config: EvaluateConfig) -> None:
        self.config = config

    def _valid_generator(self):
        
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.3
        )

        data_flow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset = "validation",
            shuffle= False,
            **data_flow_kwargs
        )


    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)


    def save_scores(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"), data = scores)



        

In [20]:
try:
    config = ConfigManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_scores()

except Exception as e:
    raise e

[2024-10-04 00:48:21,974 : INFO: common: yaml file config\config.yaml loaded successfully]
[2024-10-04 00:48:21,988 : INFO: common: yaml file params.yaml loaded successfully]
[2024-10-04 00:48:21,994 : INFO: common: Created directory at : artifacts]


Found 1090 images belonging to 7 classes.
11/11 [==============================] - 180s 16s/step - loss: 19.3770 - accuracy: 0.1835
[2024-10-04 00:51:24,376 : INFO: common: json file saved at :scores.json]
